In [ ]:
#load packages for webscraping & descriptive statistics
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as soup
import csv

In [ ]:
#link,parse, and load Ohio Cities and Counties data

wbpg = requests.get("https://www.corragroup.com/ohio-county-lookup.html")
tbl_content = soup(wbpg.content,"html.parser")

#create dataframe based on wbpg table
df_list = pd.read_html(wbpg.text)
df = df_list[0]

In [ ]:
#remove Zip code column
df2 = df.drop(columns=['ZIP Code'], axis=1)

#Add State name column
df2.insert(2,'State','OHIO')

#format all columns into Proper Case format
df2[['City','County','State']] = df2[['City','County','State']].astype(str).apply(lambda col: col.str.title())

#Updating County names for those Cities with incorrect County Names:
#Chagrin Falls (Cuyahoga), Cincinnati(Hamilton), Danville(Knox), Dayton(Montgomery), 
#Fairfax(Hamilton), Laurelville(Hocking), Loveland(Hamilton), Northwood(Wood), 
#Riverside(Montgomery), Dillonvale(Jefferson), Lima(Allen)

df2.loc[(df2['City']=='Chagrin Falls')] = df2.loc[(df2['County']=='Cuyahoga')]
df2.loc[(df2['City']=='Cincinnati')] = df2.loc[(df2['County']=='Hamilton')]
df2.loc[(df2['City']=='Danville')] = df2.loc[(df2['County']=='Knox')]
df2.loc[(df2['City']=='Dayton')] = df2.loc[(df2['County']=='Montgomery')]
df2.loc[(df2['City']=='Fairfax')] = df2.loc[(df2['County']=='Hamilton')]
df2.loc[(df2['City']=='Laurelville')] = df2.loc[(df2['County']=='Hocking')]
df2.loc[(df2['City']=='Loveland')] = df2.loc[(df2['County']=='Hamilton')]
df2.loc[(df2['City']=='Northwood')] = df2.loc[(df2['County']=='Wood')]
df2.loc[(df2['City']=='Riverside')] = df2.loc[(df2['County']=='Montgomery')]
df2.loc[(df2['City']=='Dillonvale')] = df2.loc[(df2['County']=='Jefferson')]
df2.loc[(df2['City']=='Lima')] = df2.loc[(df2['County']=='Allen')]

#Remove null city records
df2 = df2.dropna(subset=['City'])

#convert clean file to .csv
#df2.to_csv('city_county_v2.csv',index=False)


In [ ]:
#read crime dataset .csv file using pandas
crime_dataset = pd.read_csv("crime_data_files_merged.csv")

#create df of crime dataset .csv file
crime_df = pd.DataFrame(crime_dataset)

# add conditional statement to identify size of city
conditions = [
    (crime_df['Population'] <=100000),
    (crime_df['Population'] >100000) & (crime_df['Population'] <=400000),
    (crime_df['Population'] >400000)
]

# Assign city size based on population: Small city < 100K,
# Medium city >100K & <400K, Large city >400K
conditional_values = ['small', 'medium', 'large']

#create City Size column in crime_df
crime_df['City_size'] = np.select(conditions,conditional_values)    

In [ ]:
#Inner Join city/county df2 to crime datasets using city as the key
merge_df = pd.merge(crime_df,df2, how='inner', on=['City'])

#export results to csv files
# merge_df.to_csv('master_crime_data.csv', index=False)

In [ ]:
#Summarize the Crime Categories Total by Year
crime_cat_year = merge_df.groupby(['Crime_category', 'Crime_year'])['Crime_count'].sum()
print(crime_cat_year)

In [ ]:
#Summarize Crime Type Total by Sorted Descending Year
crime_type_year = merge_df.groupby(['Crime_type','Crime_year'])['Crime_count'].sum()
print(crime_type_year.sort_index(ascending=False,))

In [ ]:
#Counting highest crime type for each year

crime_stats = merge_df.groupby(['Crime_year', 'Crime_type'])['Crime_count'].sum()
print(crime_stats.sort_values(ascending=False))

In [ ]:
#Group  city size, crime year and types
crime_city_size = merge_df.groupby(['City_size', 'Crime_year', 'Crime_type'])['Crime_count'].sum()
print(crime_city_size.sort_values(ascending=False))

crime_city_size.to_csv('crime_city_size.csv')


In [ ]:
""" #Create file name and write my data into a file and export it to .csv
file_name = "OH_City_County_State_List.csv"
with open(file_name, "w", newline="") as file:
    w = csv.DictWriter(file,["city", "county", "state"])

#Export results to a csv file without the index column
df2.to_csv(file_name,index=False) """